In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
import sqlite3
import astroquery
from astropy import units as u
from astropy import coordinates as coords
from astroquery.simbad import Simbad

In [2]:
conn = sqlite3.connect("/Users/semyeong/data/cms.db")

In [3]:
query = """
select *
from star
left join tgas
  on star.tgas_row = tgas.[index]
where star.group_id == 10"""
df = pd.read_sql(query, conn)

In [4]:
print(len(df))

29


In [5]:
df.head()

,index,tgas_row,row_id,tgas_source_id,tgas_ra,tgas_dec,tgas_parallax,tgas_distance,tgas_gmag,tmass_jmag,...,group_id,group_size,index,ra,dec,parallax,pmra,pmdec,hip,tycho2_id
0,931,366159,931,1577006815932416128,195.819469,57.315207,10.245887,97.854279,8.939998,8.033,...,10,29,366159,195.819469,57.315207,10.245887,-17.102634,-8.196426,63702.0,
1,1687,371817,1687,1611685687553578752,214.072565,58.389402,9.340722,107.483816,8.402097,7.698,...,10,29,371817,214.072565,58.389402,9.340722,-16.254402,-2.902562,69721.0,
2,1888,363433,1888,1559292481018181632,205.978118,52.064391,10.737181,93.477575,6.050999,5.953,...,10,29,363433,205.978118,52.064391,10.737181,-18.270104,-5.604706,67005.0,
3,2089,363738,2089,1561366778423093248,207.114201,54.042699,10.709244,93.590625,10.724721,9.433,...,10,29,363738,207.114201,54.042699,10.709244,-18.287732,-3.933854,NaN,3851-600-1
4,2096,363746,2096,1561439689788561664,206.648437,54.432659,10.333316,97.710103,5.722788,5.707,...,10,29,363746,206.648437,54.432659,10.333316,-18.532996,-4.750367,67231.0,


In [6]:
df['tycho2_id'] = df.tycho2_id.astype('str')

In [7]:
df['object_id'] = ['hip {:.0f}'.format(row.hip) if row.hip > 0
                   else 'tyc {}'.format(row.tycho2_id) for i, row in df.iterrows()]

In [58]:
customSimbad = Simbad()
customSimbad.add_votable_fields('sptype')
customSimbad.add_votable_fields('orv')
customSimbad.add_votable_fields('bibcodelist(1850-2017)')

In [52]:
customSimbad.list_votable_fields()

--NOTES--

1. The parameter filtername must correspond to an existing filter. Filters include: B,V,R,I,J,K.  They are checked by SIMBAD but not astroquery.simbad

2. Fields beginning with rvz display the data as it is in the database. Fields beginning with rv force the display as a radial velocity. Fields beginning with z force the display as a redshift

3. For each measurement catalog, the VOTable contains all fields of the first measurement. When applicable, the first measurement is the mean one. 

Available VOTABLE fields:

          col0                   col1              col2     
------------------------ -------------------- --------------
      bibcodelist(y1-y2) fluxdata(filtername)       plx_qual
                     cel                 gcrv             pm
                    cl.g                  gen     pm_bibcode
                coo(opt)                   gj   pm_err_angle
             coo_bibcode                 hbet    pm_err_maja
           coo_err_angle                

NoneType

In [53]:
for field in ['rv_value', 'rvel', 'gcrv', 'orv']:
    customSimbad.get_field_description(field)

Radial velocity value. Eventually translated from a redshift
Radial velocities of galaxies
General Catalogue of Radial Velocities
oRV stellar radial velocities (will shortly replace 
              the GCRV measurements.


In [59]:
result = customSimbad.query_objects(df.object_id)

In [62]:
result['MAIN_ID', 'oRV_RVel',
 'oRV_Q',
 'oRV_Nmes',
 'oRV_Rem',
 'oRV_Orig',
 'oRV_Dis',
 'oRV_bibcode',][result['oRV_RVel'].mask==False]

MAIN_ID,oRV_RVel,oRV_Q,oRV_Nmes,oRV_Rem,oRV_Orig,oRV_Dis,oRV_bibcode
,km / s,,,,,a / mm,
object,float32,str1,int32,object,str2,int32,str19
HR 5169,-6.5,C,10,,##,0,1979IAUS...30...57E
* 84 UMa,-1.8,B,47,,##,0,1979IAUS...30...57E
* 81 UMa,-7.0,D,25,,##,0,1979IAUS...30...57E
HD 124370,4.0,C,2,,##,0,1953GCRV..C......0W
HR 5345,-15.0,C,4,,##,0,1953GCRV..C......0W
HR 5372,-3.2,B,4,,##,0,1953GCRV..C......0W
HD 133909,-9.0,D,6,,##,0,1953GCRV..C......0W
HR 5280,-9.5,C,10,,##,0,1979IAUS...30...57E


In [10]:
result.sort('BIBLIST_1850_2017')
result['MAIN_ID', 'SP_TYPE', 'BIBLIST_1850_2017']

/Users/semyeong/anaconda2/envs/gwb/lib/python3.5/site-packages/astropy/table/column.py:1096: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  ma.MaskedArray.__setitem__(self, index, value)


MAIN_ID,SP_TYPE,BIBLIST_1850_2017
object,object,int32
TYC 3860-1483-1,,0
TYC 3471-233-1,,0
TYC 4173-609-1,,0
TYC 3851-600-1,,0
TYC 3867-1373-1,,0
HD 234061,G0,0
HD 234065,G5,0
TYC 3875-762-1,,0
HD 238351,G0,0


In [11]:
import sys
sys.path.append('../')
import tap

class SimbadTAP(tap.TAP_Service):
    def __init__(self, *args, **kwargs):
        host = "simbad.u-strasbg.fr"
        port = 80
        path = "/simbad/sim-tap"
        tap.TAP_Service.__init__(self, host, path, port, *args, **kwargs)

In [18]:
query_template = """
SELECT BIBCode,
       Journal,
       Title,
       Year,
       Volume,
       DOI
FROM ref JOIN has_ref ON oidbibref = oidbib JOIN ident ON has_ref.oidref = ident.oidref
WHERE id = '{}'
ORDER BY Year DESC;
"""

In [19]:
mytap = SimbadTAP()

In [47]:
mytap.get_table_info('ref')

oidbib,bibcode,year,journal,page,last_page,volume,title,doi,nbobject
int64,object,int16,object,int32,int32,int32,object,object,int32


In [24]:
import time

In [25]:
biblist = []
for iden in result['MAIN_ID']:
    print('searching for {}'.format(iden.decode()))
    biblist.append(mytap.query(query_template.format(iden.decode())))
    time.sleep(5)  ## seems that I'm violating some rate limit or something

searching for TYC 3860-1483-1
searching for TYC 3471-233-1
searching for TYC 4173-609-1
searching for TYC 3851-600-1
searching for TYC 3867-1373-1
searching for HD 234061
searching for HD 234065
searching for TYC 3875-762-1
searching for HD 238351
searching for TYC 3868-177-1
searching for TYC 4174-1117-1
searching for BD+51  1966
searching for TYC 3861-1374-1
searching for BD+60  1587
searching for BD+56  1762
searching for HD 113491
searching for HD 125259
searching for HD 133909
searching for NAME Stella Ludoviciana
searching for HD 135501
searching for HD 125557
searching for HD 124370
searching for HD 129865
searching for HR  5345
searching for HR  5280
searching for HR  5372
searching for HR  5169
searching for *  81 UMa
searching for *  84 UMa


In [27]:
from astropy import table

In [31]:
dfBib = pd.concat([t.to_pandas() for t in biblist])

In [39]:
# add which object's reference it is
for i, t in enumerate(biblist):
    t['obj'] = [df.object_id[i] for j in range(len(t))]

In [41]:
tbib = table.vstack(list(filter(lambda x: len(x)>0, biblist)))

In [43]:
tbib['obj', 'title'].show_in_browser()

In [45]:
tbib.write('biblist_group10.csv')

In [47]:
tbib.write('biblist_group10.html')

In [48]:
!open biblist_group10.html

In [33]:
dfBib.title

0      b'X-ray-emitting stars identified from the ROS...
0      b'Radial velocities for the HIPPARCOS-Gaia Hun...
1      b'Fundamental parameters and infrared excesses...
2      b'Bayesian inference of stellar parameters and...
0      b'Radial velocities for the HIPPARCOS-Gaia Hun...
1      b'Fundamental parameters and infrared excesses...
2      b'The Tokyo PMC Catalog 90-93: Catalog of posi...
0      b'Radial velocities for the HIPPARCOS-Gaia Hun...
1      b'Fundamental parameters and infrared excesses...
2      b'Bayesian inference of stellar parameters and...
0      b'Bright 22 {mu}m excess candidates from the w...
1      b'Radial velocities for the HIPPARCOS-Gaia Hun...
2      b'Fundamental parameters and infrared excesses...
3      b'A catalogue of young runaway Hipparcos stars...
4      b'The Tokyo PMC Catalog 90-93: Catalog of posi...
5      b'Vitesses radiales. Catalogue WEB: Wilson Eva...
6      b'UBV photometry of stars whose positions are ...
0      b'Parallactic motion for

In [22]:
print(query_template.format(iden.decode()))


SELECT BIBCode,
       Journal,
       Title,
       Year,
       Volume,
       DOI
FROM ref JOIN has_ref ON oidbibref = oidbib JOIN ident ON has_ref.oidref = ident.oidref
WHERE id = 'HD 125259'
ORDER BY Year DESC;



In [52]:
result = mytap.query(query)

In [54]:
result.show_in_browser()